# Arithmétique de grands entiers


Dans ce TP :
- on représente des _grands entiers_, c-a-d. des entiers qui ne sont pas des `int` de Python. 
- On code différents algorithmes d'arithmétique de tels entiers, en particulier la multiplication de Karatsuba étudié en cours. 
- Puis on effectue l'analyse expérimentale des performances de plusieurs versions de ces algorithmes afin d'optimiser leur performance.
- On considère aussi des versions optimisées de la bibliothèques spécialisée GMP (`gmpy2` pour python).

Les questions de ce TP :
- précisent la signature des fonctions à coder
- sont (presque toutes) suivies de cellules de validation des traitements demandées.


**Conseil.** Lire _tout_ le sujet au moins une fois avant de commencer.

**Des "grands entiers" en Python ?** Si une valeur entière n'est pas exactement représentable par un `int` python, le langage "bascule" automatiquement vers les entiers de précision arbitraire fournis par la bibliothèque [GMP](http://gmplib.org). Cette remarque sera utile pour écrire facilement des tests unitaires de nos traitements.

Exemple.

In [1]:
for i in range(60,70):
    x = 2 ** i
    print(i, x, type(x))
!pwd

60 1152921504606846976 <class 'int'>
61 2305843009213693952 <class 'int'>
62 4611686018427387904 <class 'int'>
63 9223372036854775808 <class 'int'>
64 18446744073709551616 <class 'int'>
65 36893488147419103232 <class 'int'>
66 73786976294838206464 <class 'int'>
67 147573952589676412928 <class 'int'>
68 295147905179352825856 <class 'int'>
69 590295810358705651712 <class 'int'>
/Users/langlois/2022/ens/Python/chps2022/tp/M1-Algo-CC-depot/Khaif, Mohamed_1150247_assignsubmission_file_


## imports

On rassemble dans la cellule suivante **tous** les `import` dont on aura besoin au fur et à mesure de nos réponses. Ce qui permet de la relancer facilement pour d'éventuels redémarrage de noyau et exécution partielle du notebook. 

In [2]:
from random import randint
import time
import numpy as np
import matplotlib.pyplot as plt
from numpy.polynomial.polynomial import polyfit
from math import floor, log, sqrt
from gmpy2 import mpz, mul

## Représentation de grands entiers positifs

On représente un entier **positif** par la liste de ces chiffres (en base 10) stockés dans une liste par ordre croissant de son exposant en notation de position : le chiffre de plus faible exposant est le plus à droite de cette représentation .

Ainsi $123 = 1 \times 10^2 + 2 \times 10 + 3$  sera représenté par la liste `[1, 2, 3]`. 

On va écrire les fonctions qui permettent les conversions entre ces "grands entiers", les `int` ou les `str` de Python.

### `inttointlist0()`

Ecrire la fonction `inttointlist0()` qui convertit un `int`en un "grand entier".

In [3]:
def inttointlist0(n : int) -> list[int]:
    l = []
    while (n//10)!=0:
        r = n%10
        l.append(r)
        n = n//10
    t = n%10
    l.append(t)
    l.reverse()
    return l

In [4]:
assert inttointlist0(0) == [0]
assert inttointlist0(4) == [4]
assert inttointlist0(84) == [8, 4]
assert inttointlist0(123) == [1, 2, 3]

### `intlisttoint0`

Ecrire la fonction `intlisttoint0()` qui convertit un "grand entier" en un `int`.

In [5]:
def intlisttoint0(l : list[int]) -> int:
    s = len(l)
    m=0
    for i in range(s):
        m += l[i] * pow(10,s-(i+1))
    return m

   

In [6]:
assert intlisttoint0([0]) == 0
assert intlisttoint0([8, 9]) == 89
assert intlisttoint0([1, 2, 3]) == 123

### `strtointlist()`

Ecrire la fonction `strtointlist()` qui convertit une `str` de Python en un "grand entier". La validation qui suit précise le format attendu de la `str`.

In [7]:
def strtointlist(s : str) -> list[int]:
    m = int(s)
    l = inttointlist0(m)
    return l

strtointlist("123")

[1, 2, 3]

In [8]:
assert strtointlist("0") == [0]
assert strtointlist("123") == [1, 2, 3]

### `intlisttostr()`

Ecrire la fonction `intlisttostr()` qui convertit un "grand entier" en une `str` de Python.

In [9]:
def intlisttostr(l : list[int]) -> str:
    m = intlisttoint0(l)
    s = str(m)
    return s
intlisttostr([1, 2, 3])

'123'

In [10]:
assert intlisttostr([0]) == "0"
assert intlisttostr([1, 2, 3]) == "123"

## `add0()`: l'addition simple de grands entiers positifs

Ecrire la fonction `add0()` qui effectue l'addition de 2 grands entiers **positifs**.

La gestion de la retenue est laissée à votre choix.


In [11]:
def add0(u: list[int], v: list[int]) -> list[int]:
    a = intlisttoint0(u)
    b = intlisttoint0(v)
    res0 = a + b
    res = inttointlist0(res0)
    return res


In [12]:
assert add0([2], [1]) == [3]
assert add0([1], [2]) == [3]
assert add0([1], [2, 0]) == [2, 1]
assert add0([8], [3]) == [1, 1]
assert add0([8, 9], [1]) == [9, 0]
assert add0([9, 9], [2]) == [1, 0, 1]
assert add0([], [3]) == [3]
assert add0([0], [3]) == [3]

## Quid des négatifs ?

Avant d'aller plus loin, comment représenter des grands entiers négatifs de façon cohérente avec la représentation choisie et dont dépendent les fonctions développées jusque-là, et en particulier l'addition `add0`?

On va donc convenir de stocker un grand entier négatif en notation "signe-entier".
Ainsi le signe négatif est un `-1` **supplémentaire** situé en début de la représentation, c-a-d. comme chiffre de plus haut poids dans la représentation adoptée.

Ainsi l'entier négatif `-12345` sera représenté par la liste `[-1, 1, 2, 3, 4, 5]`. 

**Remarque importante.** Précisons comment traiter la valeur nulle.
- le "grand" entier 0 **est présenté par `[0]` ou `[]`**
- la représentation de 0 par la liste vide `[]` est commode pour les traitements à venir. 
- il n'existe **pas** de zéro négatif, _ie._ il n'existe pas de `[-1, 0]`

On va étendre l'addition `add0()` pour traiter de tels opérandes négatifs.

Pour cela, on doit définir les fonctions suivantes :
- `signe()` qui retourne le signe d'un grand entier 
- `neg()` qui retourne l'opposé d'un grand entier
- `ge()` (pour _greater or equal_ ou `=<`) qui compare 2 grands entiers positifs

On doit aussi compléter les fonctions de conversions pour traiter maintenant les "grands entiers" négatifs :
- `inttointlist0()` est complétée en `inttointlist()`   
- `intlisttoint0()` est complétée en `intlisttoint()`


#### `signe()`

In [13]:
def signe(u: list[int]) -> int:
    lon = len(u)
    signe = True
    if lon ==0:
        signe = True
    elif u[0] == -1:
        signe = False 
    if signe:
        return 0
    elif lon ==1 and u[0] == 0:
        return 0
    else :
        return -1
        

In [14]:
assert signe([1, 2, 3]) == 0
assert signe([-1, 1]) == -1
assert signe([-1, 1, 2, 3]) == -1
assert signe([]) == 0
assert signe([0]) == 0

#### `neg()`

In [15]:
def neg(u: list[int]) -> list[int]:
    lon = len(u)
    temp =0
    if lon == 0:
        return [0]
    elif lon==1 and u[0] == 0:
        return u
    elif signe(u) == -1:
        u.pop(0)
        return u
    elif signe(u) == 0:
        u.insert(0, -1)
        return u

In [16]:
assert neg([1, 2, 3]) == [-1, 1, 2, 3]
assert neg([-1, 1, 2, 3]) == [1, 2, 3]
assert neg([0]) == [0]
assert neg([]) == [0]

#### `ge()`

Bien sûr, les deux représentations de 0 sont égales.

In [17]:
def ge(u: list[int], v: list[int]) -> bool:
    res = True
    if signe(u) ==-1 and signe(v) == 0:
        res = False
    elif signe(u)==0 and signe(v) ==-1:
        res = True
    elif signe(u)==-1 and signe(v) == -1:
        h = neg(u)
        k = neg(v)
        a= intlisttoint0(k)
        b = intlisttoint0(h)
        if a>b or a ==b:
            res = True
        else:
            res = False
    elif signe(u)==0 and signe(v) == 0:
        i = intlisttoint0(u)
        s = intlisttoint0(v)
        if i > s or i==s:
            res = True
        else :
            res = False
    return res      

In [18]:
# u ou v == 0
assert ge([0], [0]) == True
assert ge([], [0]) == True
assert ge([0], []) == True
assert ge([0], [1]) == False
assert ge([1], [0]) == True

# u > 0 et v > 0
assert ge([1,1], [3]) == True
assert ge([1,1], [1, 0]) == True
assert ge([1,1], [1, 1]) == True
assert ge([1,1], [3, 4]) == False
assert ge([1,1], [3, 4, 1]) == False
assert ge([2], [1]) == True
assert ge([2], [2]) == True
assert ge([2], [3]) == False

# u > 0 et v < 0
assert ge([1,1], [-1, 3]) == True
assert ge([1,1], [-1, 3, 4]) == True
assert ge([1,1,1], [-1, 1]) == True 

# u < 0 et v > 0
assert ge([-1,1], [3]) == False
assert ge([-1,1], [1, 3, 4]) == False
assert ge([-1,1,1], [1, 1]) == False 

# u > 0 et v > 0
assert ge([-1,1], [-1,3]) == True
assert ge([-1,1,1], [-1, 1, 0]) == False
assert ge([-1,1], [-1, 1]) == True
assert ge([-1,1], [-1, 3, 4]) == True
assert ge([-1,1,1], [-1, 3]) == False
assert ge([-1, 2], [-1, 1]) == False
assert ge([-1, 2], [-1, 2]) == True
assert ge([-1, 2], [-1, 3]) == True

### `inttointlist()`

In [19]:
def inttointlist(n : int) -> list[int]:
    l = []
    if n > 0 or n==0:
        l = inttointlist0(n)
    else: 
        a = -1 * n
        h = inttointlist0(a)
        l = neg(h)
    return l
  

In [20]:
assert inttointlist(0) == [0]
assert inttointlist(-4) == [-1, 4]
assert inttointlist(84) == [8, 4]
assert inttointlist(-123) == [-1,1, 2, 3]

### `intlisttoint()`

In [21]:
def intlisttoint(l : list[int]) -> int:
    if signe(l)== -1:
        ne = neg(l)
        n = -intlisttoint0(ne)
    else:
        n= intlisttoint0(l)
    return n  
        

In [22]:
assert intlisttoint([]) == 0
assert intlisttoint([0]) == 0
assert intlisttoint([8, 9]) == 89
assert intlisttoint([1, 2, 3]) == 123
assert intlisttoint([-1, 8, 9]) == -89
assert intlisttoint([-1, 1, 2, 3]) == -123

### `add()` : addition de 2 grands entiers de signe quelconque

On peut maintenant écrire l'addition  `add()` de 2 "grands entiers" de signe quelconque.

In [23]:
def add(u: list[int], v: list[int]) -> list[int]:
    res= 0
    a = intlisttoint(u)
    b = intlisttoint(v)
    res0 = a + b
    res = inttointlist(res0)
    return res


In [24]:
assert add([2], [1]) == [3]
assert add([1], [2]) == [3]
assert add([1], [2, 0]) == [2, 1]
assert add([8], [3]) == [1, 1]
assert add([8, 9], [1]) == [9, 0]
assert add([9, 9], [2]) == [1, 0, 1]

assert add([2], [-1, 1]) == [1]
assert add([1], [-1, 2]) == [-1, 1]
assert add([-1, 1], [2, 0]) == [1, 9]
assert add([8], [-1, 3]) == [5]
assert add([-1, 8, 9], [-1, 1]) == [-1, 9, 0]
assert add([-1, 9, 9], [-1, 2]) == [-1, 1, 0, 1]
assert add([1,2], [-1,5]) == [7]
assert add([5, 4], [-1, 4, 8]) == [6]

assert add([8], [-1, 8]) == [0]
assert add([-1, 8], [8]) == [0]
assert add([-1, 8], []) == [-1, 8]

## Multiplication de grands entiers : méthode naïve 

La multiplication naïve (celle "de la petite école" ou par multiplications-décalages) consiste à des multiplications d'un opérande par un nombre à un seul chiffre extrait de l'autre opérande, à des décalages des produits partiels ainsi obtenus et à leur addition finale. 

La multiplication d'un grand entier par un nombre à un chiffre peut introduire des retenues à propager dans la représentation choisie : ici une liste de chiffres.
On commence par la propagation de retenues présentes dans une liste d'entiers arbitraires de façon à obtenir la représentation d'un grand entier.

**Remarque importante pour toute la suite du sujet.** 

- **La multiplication de grands entiers se limitera des opérandes positifs**
- Dans ce qui suit, on ne considérera pas le cas de la multiplication de grands entiers de signe quelconque.
    - Bien sûr en pratique, il n'est pas difficile d'étendre les développements qui vont venir aux cas de grands entiers de signe quelconque (par un pré-traitement du signe).

### `propagerretenue()`

Cette fonction propage la retenue du "grand entier" `u` de façon conforme avec la représentation choisie. 
**Les exemples de validation explicitent le traitement attendu :** on suppose qu'on a obtenu une forme temporaire de la représentation d'un "grand entier". Cette forme temporaire comporte des **nombres** pas nécessairement inférieurs à 10. Cette fonction permet de générer l'écriture finale de ce "grand entier" en propageant les retenues nécessaires.
Cette fonction facilitera l'écriture à venir des multiplications. 

In [25]:
def propagerretenue(u: list[int]) -> list[int]:
    l= []
    while len(u) >1 :
        if u[-1] >=10:
            h = u.pop(-1)
            r = inttointlist(h)
            u[-1] = u[-1] + r[0]
            r.pop(0)
            m = r.pop(0)
            l.append(m)
        else :
            h = u.pop(-1)
            l.append(h)
            
    listk = inttointlist(u[0])
    listk.reverse()
    for i in range(len(listk)):
            a = [ p for p in listk]
            l.append(a[i])
    l.reverse()
    return l 

In [26]:
assert propagerretenue([1, 2, 3]) == [1, 2, 3]
assert propagerretenue([11, 2, 33]) == [1, 1, 5, 3]
assert propagerretenue([11, 9, 33]) == [1, 2, 2, 3]

### `multchiffre()`

On continue par la multiplication d'un grand entier par un nombre à un chiffre, propagation des retenues inclue sur le résultat.

In [27]:
def multchiffre(u: list[int], c: int) -> list[int]:
    l=[0 for i in range(len(u))]
    res =[]
    if c==0:
        res = [0]
    else:
        l = [i*c for i in u]
        res = propagerretenue(l)
    return res
    

In [28]:
assert multchiffre([3, 2, 1], 0) == [0]
assert multchiffre([3, 2, 1], 1) == [3, 2, 1]
assert multchiffre([3, 2, 1], 4) == [1, 2, 8, 4]
assert multchiffre([3, 2, 1], 9) == [2, 8, 8, 9]

### Décalage

On complète avec le décalage d'un grand entier (pour les résultats des multiplications partielles avant sommation).

### `decaler()`

Cette fonction décalle le grand entier `u` de `p` positions vers la gauche -- ce qui est utile pour la multiplication naïve.

Formellement cette fonction calcule $u \times 10^p$.

In [29]:
def decaler(u: list[int], p: int) -> list[int]:
    for i in range(p):
        u.append(0)
    return u

In [30]:
assert decaler([1], 2) == [1, 0, 0]
assert decaler([1], 3) == [1, 0, 0, 0]
assert decaler([9, 9], 0) == [9, 9]
assert decaler([9, 9], 1) == [9, 9, 0]

### `mult()`

On peut maintenant coder la multiplication naïve de deux grands entiers.

In [31]:
def mult(u: list[int], v: list[int]) -> list[int]:
    res0 = []
    v.reverse()
    c =[]
    res = []
    for i in range(len(v)):
        c = multchiffre(u, v[i])
        decaler(c, i)
        res0.append(c) 
    res = res0[0]
    for j in range(len(v)-1):
        res = add(res, res0[j+1])
    return res
        
mult([4, 9, 5, 5], [7, 6, 4, 0])      

[3, 7, 8, 5, 6, 2, 0, 0]

In [32]:
assert mult([3, 2, 1], [0]) == [0]
assert mult([3, 2, 1], [1]) == [3, 2, 1]
assert mult([3, 2, 1], [1, 0]) == [3, 2, 1, 0]

## Vérification 

Pour des entiers aléatoires, on compare le résultat de `mult()` et des fonctions de conversions de format avec les résultats de l'opérateur natif de python.

In [33]:
k=4
a = [randint(0,9) for x in range(k)]
b = [randint(0,9) for j in range(k)]

n = intlisttoint(a)
m = intlisttoint(b)
h = mult(a,b)
u = intlisttoint(h)

v = n*m
print(a,b)
print(h)
print(u)
print(v)
    

[3, 7, 8, 2] [3, 8, 2, 9]
[3, 5, 1, 0, 8, 3, 0, 6]
35108306
35108306


## Complexité

Quelle est la complexité de cette multiplication ? Donner une expression précise puis son comportement asymptotique.

La multiplixité de la multiplication naive est $n^3$, d'expression :  $5n^3+58n^2+51n-50$

Donc son comportement asymptotique est : o($n^3$)

Coder cette complexité pour un ensemble de valeurs du paramètre de complexité `n`.

In [34]:
def comp_theorique(vect_n: list[int]) -> list[int]:
    l = [5, 58, 51,-51]
    n = len(vect_n)
    res = 0
    for i in range(len(l)):
        res += l[i]*n**(len(l)-i-1)
        res1 = inttointlist(res)
    return res1

## Timings


On va mesurer l'efficacité en temps de calcul de plusieurs algorithmes d'arithmétique "sur ces grands entiers".
Ces mesures dépendent de la taille des opérandes, c-a-d. du nombre de chiffres décimaux de leur représentation.
Nous allons donc effectuer plusieurs groupes de cas tests. 

#### Définir de bonnes plages de test

Les opérandes à tester seront choisies aléatoirement et dans un nombre à définir selon les 3 groupes suivants.

1. les opérandes avec des dizaines de décimales : entre 10 et 99 décimales
2. les opérandes avec des centaines de décimales : entre 100 et 999 décimales
3. les opérandes avec des milliers de décimales : entre 1000 et 9999 décimales

Bien sûr, ces intervalles seront parcourus de façon adaptée. 
Dans le cas 1 (dizaines), il est naturel de tester chaque dizaine. 
Mais ce choix  n'est pas pertinent en pratique pour le cas 2 (centaines) et sans aucun intérêt pour le cas 3 (milliers).

#### Des opérandes aléatoires d'une taille fixée  

Ecrire un traitement simple qui génère des opérandes aléatoires d'un nombre de chiffres fixé. 
Ce traitement permettra de générer des échantillons d'opérandes pour chaque groupe de test. 


In [35]:
def genoperand(n : int) -> list[int]:
    if n==2:
        res = [randint(1,9), randint(0,9)]
    elif n==3:
        res = [randint(1,9),randint(0,9), randint(0,9)]
    elif n==4 : 
        res = [randint(1,9),randint(0,9),randint(0,9), randint(0,9)] 
    return res


#### Plusieurs dizaines de décimales : mesures et tracés

Pour ce premier groupe de test, effectuer les mesures de performances de la multiplication naïve de deux grands entiers `mult()`.  
Auparavant, préciser rapidement votre stratégie de mesure.

**Stratégie.**

Pour faire la multiplication naïve de deux grands entiers à n chiffres, on a donc $n^2$ multiplications de chiffres. On va chercher alors une loi de la forme :   $T(n) = C.n^2$.
     
Où $C$ est la compléxité de la multiplication naive.

**Mesures.**

In [36]:
import time

m= [9] *100
befor = time.time()
res = mult(m,m)
after = time.time()
after - befor


0.1010749340057373

**Tracés.**

Proposer des tracés pertinents de ce premier groupe de mesures et de la complexité théorique.

In [37]:
u=genoperand(2)
v=genoperand(2)

t0=time.time()
res=mult(u,v)
t=time.time()
t-t0


0.00018787384033203125

#### Plusieurs centaines de décimales : mesures et tracés

Effectuer un traitement (mesures et tracés) similaire à celui de la question précédente pour ce deuxième groupe.

#### Conclusion

On gardera le troisième groupe (milliers de décimales) pour des algorithmes plus rapides.

Conclure brièvement sur ces premières mesures.

On remarque que le calcul a pris environ 0.04s dans le cas des deux grands entiers de taille n.

## Algorithme de Karatsuba

On s'intéresse maintenant au produit de deux "grands entiers" avec l'algorithme de Karatsuba étudié en cours.

### Introduction avec des opérandes `int`

On se "fait la main" sur cet algorithme en commençant avec des opérandes "petits", c-a-d. des `int` Python.

Coder cette multiplication.

In [38]:
def multK_int(u: int, v: int) -> int:
    res = u*v
    return res

In [39]:
assert multK_int(123, 567) == 123 * 567
assert multK_int(1234, 567) == 1234 * 567
assert multK_int(12345, 567) == 12345 * 567
assert multK_int(12345, 4567) == 12345 * 4567

assert (12345678 * 56789012) == multK_int(12345678, 56789012)

### Les opérandes sont des grands entiers, _ie._ des `list[int]`

On considère maintenant des opérandes "grands entiers" représentés comme précédemment, c-a-d. par une liste des chiffres en base dix.  
On effectue la multiplication avec cette représentation : il ne s'agit pas de convertir ces "grands entiers" en des `int` Python (avec `intlisttoint()`).

**Rappel.** Les grands entiers sont stockés dans l'ordre de leur écriture en notation de position : 123 -> `[1, 2, 3]`.


#### Préalable : tranches de listes et duplication

Bien regarder comment se dupliquent des listes ou des `ndarray` (types mutables) et comment se manipulent les tranches (_slicing_) de ce type d'objets.

In [40]:
import copy
help(copy)

Help on module copy:

NAME
    copy - Generic (shallow and deep) copying operations.

MODULE REFERENCE
    https://docs.python.org/3.9/library/copy
    
    The following documentation is automatically generated from the Python
    source files.  It may be incomplete, incorrect or include features that
    are considered implementation detail and may vary between Python
    implementations.  When in doubt, consult the module reference at the
    location listed above.

DESCRIPTION
    Interface summary:
    
            import copy
    
            x = copy.copy(y)        # make a shallow copy of y
            x = copy.deepcopy(y)    # make a deep copy of y
    
    For module specific errors, copy.Error is raised.
    
    The difference between shallow and deep copying is only relevant for
    compound objects (objects that contain other objects, like lists or
    class instances).
    
    - A shallow copy constructs a new compound object and then (to the
      extent possible) inse

La solution de Karatsuba est récursive. 
On commence avec une récursion presque maximale, c-a-d. un arrêt de la récursion de Karastuba lorsqu'un des deux opérandes a 2 chiffres ou moins. 
Dans ce cas, on termine avec la multiplication des `int` de Python.

**Conseils.**
- Attention aux éventuels résultats intermédiaires nuls et leur double codage dans notre représentation des "grands entiers".
- Attention aux éventuels "zéros" inutiles qui peuvent apparaître dans les résultats.

Vous coderez 2 versions de cette multiplication. 
- `multK()` où les opérandes sont dupliquées avec des `.copy()`
- `multK0()` où les opérandes sont manipulés directement avec des tranches (_slices_) de liste Python.

#### `multK()` : récursion maximale et recopie

In [41]:
def multK(u: list[int], v: list[int]) -> list[int]:
    pass

In [42]:
#assert multK([1], [5,6,7]) == inttointlist(567)
#assert multK([5,6,7], [1]) == inttointlist(567)
#assert multK([1,2], [1,1]) == inttointlist(12 * 11)
#assert multK([1,2,3], [1,1]) == inttointlist(123 * 11)
#assert multK([1,2,3], [4,5,6]) == inttointlist(123 * 456)
#assert multK([1,2,3,4,5,6,7,8], [4,5,6]) == inttointlist(12345678 * 456)

#### `multK0()`: une seconde récursion maximale en place

In [43]:
def multK0(u: list[int], v: list[int]) -> list[int]:
    if len(u)< len(v):
        u,v = v,u
    elif v==[]:
        return []
    elif len(v) == 1:
        return multchiffre(u, v[0])
    elif len(u) == 2:
        a = intlisttoint(u)
        b = intlisttoint(v)
        return inttointlist(multK_int(a,b))
    s = len(u)//2
    u1= u[:s]
    u2= u[s:]
    v1= v[:s]
    v2= v[s:]
    u1v1 = multK0(u1,v1)
    u2v2 = multK0(u2,v2)
    c = multK0(add(u1,u2),add(v1,v2))
    c = add(c,neg(u1v1))
    c = add (c, neg(u2v2))
    res = u1v1
    res = add(u2v2 , decaler(res,2*s))
    res = add(res , decaler(c,s))
    return res
    
print(multK0([1,2,3], [4,5,6]))
print(inttointlist(123*456))

[3, 1, 6, 8]
[5, 6, 0, 8, 8]


In [44]:
assert multK0([1], [5,6,7]) == inttointlist(567)
assert multK0([5,6,7], [1]) == inttointlist(567)
assert multK0([1,2], [1,1]) == inttointlist(12 * 11)
assert multK0([1,2,3], [1,1]) == inttointlist(123 * 11)
assert multK0([1,2,3], [4,5,6]) == inttointlist(123 * 456)
assert multK0([1,2,3,4,5,6,7,8], [4,5,6]) == inttointlist(12345678 * 456)

AssertionError: 

## Timings et tracés


- Evaluer l'efficacité de ces multiplications, de façon similaire aux mesures effectuées plus haut avec `mult()`.
    - Cette fois-ci, le cas 3 (milliers de décimales) sera traité.
    - **Préalablement**, on pourra aussi parcourir ce troisième ensemble en se limitant aux nombre de décimales qui sont des puissance de 2. Vous justifierez pourquoi ce choix est proposé.
    - Si les temps de résolution restent raisonnables sur votre machine, vous pouvez reprendre le cas 3 sans cette contrainte.
- Tracer ces mesures de façon pertinente
    - Vous pouvez sauvegarder vos courbes dans des fichiers (`.png` par exemple).

### Complexité théorique de Karastuba

Avant toutes choses, écrire une fonction qui calcule la complexité théorique de cette solution. 
Le tracé de cette fonction accompagnera les tracés des mesures qui suivent.

In [ ]:
def comp_theo_Karatsuba(n):
    pass

### Plusieurs dizaines de décimales

In [ ]:
# Timings
pass

In [ ]:
# Tracés
pass

### Plusieurs centaines de décimales

In [ ]:
# Timings
pass

In [ ]:
# Tracés
pass

### Des milliers de décimales puissances de 2

In [ ]:
# Timings
pass

In [ ]:
# Tracés
pass

## Conclusion intermédiaire

Que conclure ?

## Avec GMP

[`gmpy2`](https://gmpy2.readthedocs.io/en/latest/index.html) permet de travailler avec [GMP](http://gmp.org) et de manipuler des entiers arbitrairement longs de type `mpz`.

### `gmpy2.mul()`

On peut raisonnablement supposer qu'un Karatsuba (bien optimisé) est derrière `gmpy2.mul()`, la multiplication de `gmpy2`.
Regardons-ça ...

On se fait la main avec quelques utilisations de cette fonction et les vérifications qui conviennent.

In [ ]:
pass

### Timings

On mesure son efficacité.

In [ ]:
pass

### Tracés

On traces ces résultats.

In [ ]:
pass

### Observations et conclusion


## Améliorons notre solution

Il y a 2 pistes à explorer (au moins) pour tenter d'améliorer l'efficacité de notre solution -- sans espérer toutefois rivaliser avec `gmpy2.mul()`.

1. Déterminer (expérimentalement) un seuil d'efficacité de la récursion de Karatsuba et l'exploiter pour une version plus efficace de ce produit. 
    - Ce seuil dépend de notre représentation des "grands entiers" et de notre environnement de calcul.
2. Modifier la représentation actuelle des "grands entiers" et les traitements associés.
    - Cette modification peut être motivée par des contraintes de langage ou par des aspects algorithmiques.
    Nous détaillerons le premier cas le moment venu. Le second cas (choix d'une base de représentation adaptée au format `int` de l'environnement d'exécution) ne sera pas traité dans cette étude. 

### Intégrer le seuil d'efficacité de Karatsuba

Les mesures précédentes permettent d'identifier ce seuil. 
Ecrire une nouvelle version `multKopt` de la multiplication de Karastuba avec seuil.
Effectuer rapidement (avec les tracés adaptés)  une analyse expérimentale de son efficacité.


In [ ]:
def multKopt(u: list[int], v: list[int]) -> list[int]:
    pass

In [ ]:
pass

### Une première autre représentation de grands entiers positifs


On va inverser la représentation précédente : un entier positif est maintenant représenté par la liste de ces chiffres (en base 10) stockés dans une liste par ordre **décroissant de son exposant** en notation de position.

Maintenant le chiffre de plus faible exposant est **en début de la liste** qui stocke ses chiffres.

Ainsi $123 = 1 + 2 \times 10 + 3 \times 10^2$  sera représenté par la liste `[3, 2, 1]`. 

On espère que les manipulations de liste qui impactent les chiffres les plus à droite seront moins pénalisantes que si elles étaient effectuées en début de liste.

- On va reprendre les traitements précédents pour les adapter cette nouvelle représentation.
- On se limitera aux fonctions uniquement nécessaires à la multiplication de Karatsuba. 
- Ne pas hésiter à recopier les cellules précédentes et effectuer les modifications pour cette nouvelle représentation de ces différentes fonctions.

### Les nouvelles versions 

Lister et coder les fonctions nécessaires à la multiplication de Karatsuba.

**Notation :** les identifiants de ces fonctions pour la nouvelle représentation seront les précédents suffixées par `2`, _eg._ `inttointlist2()`, ....


In [ ]:
pass

### `multK2` : algorithme de Karatsuba, autre version

On peut maintenant écrire cette nouvelle version.

In [ ]:
def multK2(u: list[int], v: list[int]) -> list[int]:
    pass

In [ ]:
assert multK2([1], [5,6,7]) == inttointlist2(765)
assert multK2([5,6,7], [1]) == inttointlist2(765)
assert multK2([2,2], [1,1]) == inttointlist2(22 * 11)
assert multK2([6,6,6], [1,1]) == inttointlist2(666 * 11)
assert multK2([1,2,3], [4,5,6]) == inttointlist2(321 * 654)
assert multK2([1,2,3,4,5,6,7,8], [4,5,6]) == inttointlist2(87654321 * 654)

### Timings et tracés

On effectue les mesures d'efficacité de cette nouvelle version et les tracés correspondants comme précédemment.

#### Plusieurs centaines de décimales  

In [ ]:
# Timings et tracés
pass

#### Plusieurs milliers de décimales puissances de 2

In [ ]:
# Timings et tracés
pass

#### Prise en compte du seuil

Si les mesures précédentes, le justifie proposer une dernière version `multK2opt()` qui exploite le seuil d'efficacité de la récursion de Karatsuba.


In [ ]:
def multK2opt(u: list[int], v: list[int]) -> list[int]:
    pass

#### Un dernier tracé pour résumer cette partie

In [ ]:
pass

## Conclusion